# Examples

The `dCacheFileSystem` class provides a python file-system interface for dCache. It builds on [File-System Spec](https://filesystem-spec.readthedocs.io) (in particular, on its `HTTPFileSystem` implementation) and it can be used as an independent object or via the general-purpose `fsspec` methods.

For the following examples, the file-sytem is initially setup in the following way:

```
/
└── test
    ├── empty_testdir
    ├── testdir_1
    │   ├── file_1.txt
    │   └── file_2.txt
    └── testdir_2
        ├── file_1.txt
        └── file_2.txt
```

## Independent usage

In [1]:
from dcachefs import dCacheFileSystem

api_url = 'https://dcacheview.grid.surfsara.nl:22880/api/v1'

# read authentication token
with open('macaroon.dat') as f:
    token = f.read().strip()

fs = dCacheFileSystem(api_url=api_url, token=token)

The following methods are implemented via the [dCache API](https://dcache.org/old/manuals/UserGuide-6.2/frontend.shtml):

In [2]:
fs.ls('/test/testdir_1', detail=False)

['/test/testdir_1/file_2.txt', '/test/testdir_1/file_1.txt']

In [3]:
fs.ls('/test/testdir_1/file_1.txt')

[{'name': '/test/testdir_1/file_1.txt',
  'size': 12,
  'type': 'file',
  'created': datetime.datetime(2021, 1, 20, 9, 43, 14, 231000),
  'modified': datetime.datetime(2021, 1, 20, 9, 43, 14, 298000)}]

In [4]:
fs.ls('/test/empty_testdir')

[]

In [5]:
fs.info('/test/testdir_1/')

{'name': '/test/testdir_1/',
 'size': 512,
 'type': 'directory',
 'created': datetime.datetime(2021, 1, 20, 9, 43, 14, 7000),
 'modified': datetime.datetime(2021, 1, 20, 9, 43, 14, 522000)}

In [6]:
fs.mv('/test/testdir_2/file_1.txt', '/test/testdir_2/file_renamed.txt')
fs.ls('/test/testdir_2/', detail=False)

['/test/testdir_2/file_2.txt', '/test/testdir_2/file_renamed.txt']

In [7]:
fs.rm('/test/testdir_2/file_2.txt')
fs.ls('/test/testdir_2/', detail=False)

['/test/testdir_2/file_renamed.txt']

In [8]:
fs.exists('/test/testdir_1/file_1.txt')

True

In [9]:
fs.exists('/test/testdir_1/nonexistent_file.txt')

False

In [10]:
fs.isfile('/test/testdir_1/file_1.txt')

True

In [11]:
fs.isdir('/test/testdir_1/')

True

In [12]:
fs.created('/test/testdir_1/file_1.txt')

datetime.datetime(2021, 1, 20, 9, 43, 14, 231000)

In [13]:
fs.modified('/test/testdir_1/file_1.txt')

datetime.datetime(2021, 1, 20, 9, 43, 14, 298000)

In [14]:
fs.size('/test/testdir_1/file_1.txt')

12

In [15]:
fs.glob('/test/testdir_*/file_*.txt')

['/test/testdir_1/file_1.txt',
 '/test/testdir_1/file_2.txt',
 '/test/testdir_2/file_renamed.txt']

In [16]:
for root, _, _ in fs.walk('/test'):
    print(root)

/test
/test/testdir_2
/test/testdir_1
/test/empty_testdir


In [17]:
fs.find('/test')

['/test/testdir_1/file_1.txt',
 '/test/testdir_1/file_2.txt',
 '/test/testdir_2/file_renamed.txt']

In [18]:
fs.du('/test') # bytes

36

In [19]:
fs.checksum('/test/testdir_1/file_1.txt')

52817760817091724892189505449282471387543089338661476495573442382936696898377

The following methods, which involve reading/writing files from/to dCache, require the WebDAV door to be specified. It can be included in the URI:

In [20]:
fs.cat('https://webdav.grid.surfsara.nl:2880/test/testdir_1/file_1.txt')

b'Hello world!'

or set via a separate input argument when instantiating the file-system object:

In [21]:
webdav_url = 'https://webdav.grid.surfsara.nl:2880'
fs = dCacheFileSystem(api_url=api_url, webdav_url=webdav_url, token=token)
fs.cat('/test/testdir_1/file_1.txt')

b'Hello world!'

In [22]:
local_path = './file.txt'
fs.download('/test/testdir_1/file_1.txt', local_path)
!ls $local_path

./file.txt


In [23]:
remote_path = '/test/testdir_2/file_uploaded.txt'
fs.upload(local_path, remote_path)
fs.ls(remote_path, detail=False)

['/test/testdir_2/file_uploaded.txt']

In [24]:
with fs.open('/test/testdir_1/file_1.txt', 'rb') as f:
    content = f.read()
content

b'Hello world!'

In [25]:
path = '/test/testdir_2/file_written.txt'
with fs.open(path, 'wb') as f:
    f.write(b'Hello world!')
fs.cat(path)

b'Hello world!'

## Usage via `fsspec`

The dCache file-system class can be used via `fsspec` after registering it as a backend implementation:

In [26]:
import fsspec
# set class as default to open URIs with the https protocol  
fsspec.register_implementation('https', dCacheFileSystem)

# URI should include protocol and WeDAV door
uri = 'https://webdav.grid.surfsara.nl:2880/test/testdir_1/file_1.txt'

with fsspec.open(uri, token=token, api_url=api_url) as f:
    content = f.read()
content

b'Hello world!'

To register the class only temporarily, the `register_implementation` context manager can be used. This takes care of removing it from the `fsspec` implemention registry:

In [27]:
import dcachefs
with dcachefs.register_implementation(protocol='https'):
    with fsspec.open(uri, token=token, api_url=api_url) as f:
        content = f.read()
content

b'Hello world!'